In [1]:
from utils import TestResults,TrainSplit
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, log_loss

import numpy as np
import optuna

In [2]:
cat_feat = TrainSplit.CATEGORIES + TrainSplit.BINARY
num_feat = TrainSplit.NUMERICAL
all_feat = cat_feat + num_feat

In [3]:
import pandas as pd
data = pd.read_csv("../Data/imputed_data_float32.csv")

In [18]:
train = data[data[TrainSplit.DATE[0]] < 66]
valid  =data[data[TrainSplit.DATE[0]] == 66]

X_train = train[all_feat]
X_val = valid[all_feat]
y_train = train[TrainSplit.IS_INSTALLED[0]]
y_val = valid[TrainSplit.IS_INSTALLED[0]]

In [20]:
import gc
gc.collect()

13071

In [21]:
X_train.shape,y_train.shape

((3387880, 78), (3387880,))

In [22]:
X_val.shape,y_val.shape

((97972, 78), (97972,))

In [24]:
def objective(trail):
    params = {
        'max_depth':trail.suggest_int('max_depth',3,8),
        'learning_rate':trail.suggest_float('learning_rate',0.05,0.3),
        'n_estimators':trail.suggest_int('n_estimators',100,650),
        'tree_method':'gpu_hist',
        'objective':'binary:logistic'
    }
    # print(use_features)
    target = TrainSplit.IS_INSTALLED[0]
    model = XGBClassifier(**params)
    model.fit(X_train[use_features],y_train)
    y_pred = model.predict(X_val[use_features])
    # print(y_pred.shape,y_val.shape)
    score = log_loss(y_val,y_pred)
    # print(model.feature_names_in_)
    return score

In [25]:
use_features = all_feat
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective, n_trials=50)

[I 2023-05-27 19:55:13,235] A new study created in memory with name: install_log_loss
[I 2023-05-27 19:55:29,335] Trial 0 finished with value: 5.972361098286672 and parameters: {'max_depth': 6, 'learning_rate': 0.23064216025780576, 'n_estimators': 641}. Best is trial 0 with value: 5.972361098286672.
[I 2023-05-27 19:55:36,258] Trial 1 finished with value: 5.9346286876467484 and parameters: {'max_depth': 5, 'learning_rate': 0.13581009901863916, 'n_estimators': 162}. Best is trial 1 with value: 5.9346286876467484.
[I 2023-05-27 19:55:48,033] Trial 2 finished with value: 5.852851666697129 and parameters: {'max_depth': 4, 'learning_rate': 0.2454055085592715, 'n_estimators': 546}. Best is trial 2 with value: 5.852851666697129.
[I 2023-05-27 19:55:57,998] Trial 3 finished with value: 5.903965409217137 and parameters: {'max_depth': 5, 'learning_rate': 0.22102000485607748, 'n_estimators': 224}. Best is trial 2 with value: 5.852851666697129.
[I 2023-05-27 19:56:08,208] Trial 4 finished with val

In [46]:
params = {'max_depth': 7, 
         'learning_rate': 0.13269007660890636, 
         'n_estimators': 477,
         'tree_method':'gpu_hist',
         'objective':'binary:logistic'}

target = TrainSplit.IS_INSTALLED[0]
model = XGBClassifier(**params)
model.fit(X_train[use_features],y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.13269007660890636,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=477, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [27]:
test = pd.read_csv('../Data/test/000000000000.csv',sep='\t')

In [28]:
data = pd.read_csv("../Data/miss_combine.csv")
data.loc[data['f_30'].isnull(),'f_30'] = 3
data.loc[data['f_31'].isnull(),'f_31'] = 3
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in fmiss:
    data[f].fillna(data[f].mean(),inplace=True)
feat_num_map = dict()
for i in range(2,43):
    out = data.groupby(f'f_{i}')[TrainSplit.IS_INSTALLED[0]].mean()
    temp = dict()
    for clas,prob in out.items():
        temp[clas]=prob
    feat_num_map[f"f_{i}"] = temp

In [30]:
from tqdm import tqdm

In [31]:
for i in tqdm(range(2,43),desc="replace"):
    test[f'f_{i}'].replace(feat_num_map[f'f_{i}'],inplace=True)
    gc.collect()

replace: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:12<00:00,  3.37it/s]


In [32]:
# Unkown class probablitys are set to 0.5 as we don't know
for i in tqdm(range(2,43),desc="replace"):
    test.loc[test[f'f_{i}']>1,f'f_{i}'] = 0.5

replace: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 1781.35it/s]


In [33]:
fmiss = "f_43,f_51,f_58,f_59,f_64,f_65,f_66,f_67,f_68,f_69,f_70".split(',')
for f in fmiss:
    test[f].fillna(test[f].mean(),inplace=True)

In [47]:
install_pred = model.predict_proba(test[all_feat])

In [48]:
install_pred[:,1]

array([0.38630268, 0.17128249, 0.03216574, ..., 0.06771537, 0.11223465,
       0.10592999], dtype=float32)

In [49]:
TestResults(row_id=test['f_0'],is_click=np.random.random(install_pred[:,1].shape[0]),
            is_install=install_pred[:,1],model_name="xgb_float_all_feat",config=params)

Saved the test result to csv file as xgb_float_all_feat_2023-05-27 22:10:08.560284.csv
Saved the model config to json file as xgb_float_all_feat_2023-05-27 22:10:08.560284.json


In [39]:
from catboost import CatBoostClassifier

In [44]:
def objective_cat(trial):
    params = {
        'depth':trial.suggest_int('max_depth',3,10),
        'iterations':trial.suggest_int('iterations',100,1000),
        'learning_rate':trial.suggest_float('learning_rate',0.05,0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'boosting_type': trial.suggest_categorical('boosting_type', ['Ordered', 'Plain']),
        'task_type':'GPU',
        'silent': True
    }
    print(params)
    model = CatBoostClassifier(**params)
    model.fit(X_train[use_features],y_train)
    y_pred = model.predict(X_val[use_features])
    score = log_loss(y_val,y_pred)
    # print(model.feature_names_in_)
    gc.collect()
    return score

In [45]:
use_features = all_feat
study_install = optuna.create_study(direction='minimize',study_name='install_log_loss')
study_install.optimize(objective_cat, n_trials=50)

[I 2023-05-27 21:18:27,904] A new study created in memory with name: install_log_loss


{'depth': 4, 'iterations': 116, 'learning_rate': 0.190142909151987, 'l2_leaf_reg': 2.286103583872621, 'boosting_type': 'Ordered', 'task_type': 'GPU', 'silent': True}


[I 2023-05-27 21:18:41,245] Trial 0 finished with value: 6.029456620893724 and parameters: {'max_depth': 4, 'iterations': 116, 'learning_rate': 0.190142909151987, 'l2_leaf_reg': 2.286103583872621, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 6.029456620893724.


{'depth': 9, 'iterations': 400, 'learning_rate': 0.05781477572058101, 'l2_leaf_reg': 2.521728403447148, 'boosting_type': 'Plain', 'task_type': 'GPU', 'silent': True}


[I 2023-05-27 21:19:00,957] Trial 1 finished with value: 5.8482595011779726 and parameters: {'max_depth': 9, 'iterations': 400, 'learning_rate': 0.05781477572058101, 'l2_leaf_reg': 2.521728403447148, 'boosting_type': 'Plain'}. Best is trial 1 with value: 5.8482595011779726.


...

{'depth': 3, 'iterations': 546, 'learning_rate': 0.28436778498202314, 'l2_leaf_reg': 6.541697863941554, 'boosting_type': 'Ordered', 'task_type': 'GPU', 'silent': True}


[I 2023-05-27 22:00:50,078] Trial 39 finished with value: 5.869057009804809 and parameters: {'max_depth': 3, 'iterations': 546, 'learning_rate': 0.28436778498202314, 'l2_leaf_reg': 6.541697863941554, 'boosting_type': 'Ordered'}. Best is trial 15 with value: 5.808434662265208.


{'depth': 10, 'iterations': 756, 'learning_rate': 0.19927822505748627, 'l2_leaf_reg': 8.332224673332847, 'boosting_type': 'Ordered', 'task_type': 'GPU', 'silent': True}


[W 2023-05-27 22:03:49,143] Trial 40 failed with parameters: {'max_depth': 10, 'iterations': 756, 'learning_rate': 0.19927822505748627, 'l2_leaf_reg': 8.332224673332847, 'boosting_type': 'Ordered'} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/home2/sisodiya.bhoomendra/venvs/python3.9_global/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_23234/3002279992.py", line 13, in objective_cat
    model.fit(X_train[use_features],y_train)
  File "/home2/sisodiya.bhoomendra/venvs/python3.9_global/lib/python3.9/site-packages/catboost/core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/home2/sisodiya.bhoomendra/venvs/python3.9_global/lib/python3.9/site-packages/catboost/core.py", line 2357, in _fit
    self._train(
  File "/home2/sisod

KeyboardInterrupt: 